In [ ]:
# Load the images in total
import pandas as pd
df = pd.read_csv('/home/eo287/mnt/s3_ccta/summaries/ccta_series_headers_only.csv')

In [ ]:
# How many studies have we processed?
df["StudyFolder"].nunique()

In [ ]:
#ImageType (0008,0008)
print(df['ImageType'].value_counts())

In [ ]:
# Keep if ImageType contains ORIGINAL, PRIMARY, AXIAL
df_filtered = df[df['ImageType'].str.contains('ORIGINAL') &
                 df['ImageType'].str.contains('PRIMARY') &
                 df['ImageType'].str.contains('AXIAL')]
df_filtered["StudyFolder"].nunique()

In [ ]:
# Apply some heuristics

# keep if SliceThickness < 1 mm
df_filtered = df_filtered[df_filtered['SliceThickness'] < 0.8]
print(f"After SliceThickness filter: {df_filtered['StudyFolder'].nunique()} studies")

# keep if ContrastBolusAgent is not null
#df_filtered = df_filtered[df_filtered['ContrastBolusAgent'].notnull()]
#print(f"After ContrastBolusAgent filter: {df_filtered['StudyFolder'].nunique()} studies")

In [ ]:
# show first 6 rows of duplicated StudyFolder
pd.set_option('display.max_columns', None)
df_filtered[df_filtered.duplicated(subset=['StudyFolder'], keep=False)].sort_values(by=['StudyFolder']).head(5)

In [ ]:
df_filtered["SeriesDescription"].value_counts()

In [ ]:
import re
import numpy as np
import pandas as pd

# ---------- patterns ----------
RE_PCT = re.compile(r'(\d{1,3})\s*%')
RE_MS  = re.compile(r'(-?\d{2,4})\s*ms', flags=re.I)

EXCLUDE_PATS = [
    r'(?i)\b(smart\s*score|calcium|CAC)\b',
    r'(?i)\b(no\s*contrast|non[-\s]?contrast|w/o\b)',
    r'(?i)\b(timing|test)\s*bolus\b',
    r'(?i)\b(localizer|scout|topogram|dose|screen\s*save)\b',
    r'(?i)\b(FL\s*CHEST|CHEST\s*WITHOUT|NON\s*CONTRAST\s*CHEST|FOV\s*CHEST)\b',
    r'(?i)\b(MPR|curved|VR|VRT|3D|minIP|maxIP|avgIP)\b',
]

# Broader Siemens/“coronary” hints beyond literal "CTA with con"
INCLUDE_PATS = [
    r'(?i)\bCTA\b',
    r'(?i)\bCorCTA\b',
    r'(?i)\bCoronary\s*CTA\b',
    r'(?i)\bGATED\s*CTA\b',
    r'(?i)\bDS[_ ]?CorCTA\b',
    r'(?i)\bDS[_ ]?CorAdSeq\b',
    r'(?i)\bFl[_ ]?CorCTA\b',
    r'(?i)\bHEARTFLOW\b',
    r'(?i)\bBestDiast\b',
    r'(?i)\bRETRO\b',
    r'(?i)\b(0\.6(25)?\s*mm?)\b',
    r'(?i)\bBv\d+\b',   # vascular kernel
]

def has_any(s, pats): return any(re.search(p, str(s)) for p in pats)

# ---------- gates ----------
def is_hard_excluded(desc: str) -> bool:
    return has_any(desc, EXCLUDE_PATS)

def is_likely_coronary(desc: str) -> bool:
    # Inclusion is generous; primary filtering is via exclusions.
    return has_any(desc, INCLUDE_PATS)

def image_type_preference(itype) -> int:
    # Prefer ORIGINAL/PRIMARY/AXIAL; do not drop if missing.
    if isinstance(itype, (list, tuple)): it = set(map(str.upper, itype))
    else: it = {str(itype).upper()}
    sc = 0
    if 'ORIGINAL' in it: sc += 12
    if 'PRIMARY'  in it: sc += 10
    if 'AXIAL'    in it: sc += 8
    if 'DERIVED'  in it: sc -= 10
    if 'SECONDARY' in it: sc -= 8
    return sc

# ---------- feature extractors ----------
def parse_phase_pct(s):
    m = RE_PCT.search(str(s)); return float(m.group(1)) if m else np.nan

def parse_ms_phase(s):
    m = RE_MS.search(str(s)); return float(m.group(1)) if m else np.nan

def parse_nominal_thickness(s):
    m = re.search(r'(?<!\d)(0\.\d{1,3})\s*mm?', str(s), flags=re.I)
    return float(m.group(1)) if m else np.nan

# ---------- scoring ----------
def kernel_score(desc):
    s = str(desc); sc = 0
    if re.search(r'(?i)\bBv40\b', s): sc += 10   # vascular
    if re.search(r'(?i)\bBr4\d\b', s): sc -= 5   # bone
    if re.search(r'(?i)\bTHIN(S)?\b', s): sc += 4
    if re.search(r'(?i)\bVS\d+\b', s): sc += 3
    return sc

def fov_score(desc):
    s = str(desc); sc = 0
    if re.search(r'(?i)\bTRUE\s*STACK\b', s): sc += 8
    if re.search(r'(?i)\bHEART\s*ONLY\b', s): sc += 6
    if re.search(r'(?i)\bFULL\s*FOV\b', s):  sc -= 3
    return sc

def phase_pref_score(desc, hr_hint=None):
    pct = parse_phase_pct(desc); ms = parse_ms_phase(desc); s = str(desc)
    base = 0
    if re.search(r'(?i)\bBestDiast\b', s): base += 18
    if re.search(r'(?i)\bBestSyst\b', s): base -= 6
    if not np.isnan(ms): base += (8 if ms < 0 else -3)
    # prefer ~75%; if HR>65 and label is systolic, allow ~45%
    target = 75.0
    if hr_hint is not None and hr_hint > 65 and re.search(r'(?i)\bBestSyst\b', s):
        target = 45.0
    if not np.isnan(pct): base += 16 * np.exp(-((pct - target)**2) / (2*10**2))
    # ranges worse than single explicit phases
    if re.search(r'\d+\s*-\s*\d+\s*%', s): base -= 4
    return base

def thickness_score(desc, slice_thickness):
    v = np.nan
    if slice_thickness is not None:
        try: v = float(slice_thickness)
        except: pass
    if np.isnan(v): v = parse_nominal_thickness(desc)
    if np.isnan(v): return 0.0
    return min(2.0, max(0.0, 1.0/v)) * 10.0  # ~10 @0.5mm; ~6.7 @0.75mm

def nimg_score(n):
    try: n = int(n); return min(n, 1200) * 0.008
    except: return 0.0

def score_row(row, hr_hint=None):
    desc = row.get('SeriesDescription','')
    sc = 0
    # validity: exclude hard negatives heavily; include likely coronary positively
    sc += -300 if is_hard_excluded(desc) else 0
    sc +=  50  if is_likely_coronary(desc) else 0
    sc += image_type_preference(row.get('ImageType', []))
    sc += kernel_score(desc)
    sc += fov_score(desc)
    sc += phase_pref_score(desc, hr_hint=hr_hint)
    sc += thickness_score(desc, row.get('SliceThickness', None))
    sc += nimg_score(row.get('NumImages', None))
    # tiny bump for HEARTFLOW export
    if re.search(r'(?i)\bHEARTFLOW\b', str(desc)): sc += 3
    return sc

# ---------- main with staged fallback + diagnostics ----------
def select_best_ccta_per_study_gentle(df, hr_hint=None, verbose=True):
    req = {'StudyFolder','ImageType','SeriesDescription'}
    miss = req - set(df.columns)
    if miss: raise ValueError(f"Missing required columns: {miss}")
    d = df.copy()

    # Stage A: primary candidates = NOT hard-excluded AND likely coronary
    A_mask = (~d['SeriesDescription'].apply(is_hard_excluded)) & (d['SeriesDescription'].apply(is_likely_coronary))
    candA = d[A_mask].copy()
    candA['__score__'] = candA.apply(lambda r: score_row(r, hr_hint=hr_hint), axis=1)

    # For studies missing in A, Stage B: allow anything NOT hard-excluded, then score
    missing_studies = set(d['StudyFolder']) - set(candA['StudyFolder'])
    if missing_studies:
        candB = d[(~d['SeriesDescription'].apply(is_hard_excluded)) & (d['StudyFolder'].isin(missing_studies))].copy()
        candB['__score__'] = candB.apply(lambda r: score_row(r, hr_hint=hr_hint), axis=1)
        cand = pd.concat([candA, candB], ignore_index=True)
    else:
        cand = candA

    # Sort + pick top per StudyFolder (tie-breakers inside score; secondary keys below)
    def _slice_for_sort(desc, st):
        v = None
        try:
            if st is not None and not pd.isna(st): v = float(st)
        except: v = None
        if v is None or np.isnan(v): v = parse_nominal_thickness(desc)
        return np.inf if (v is None or np.isnan(v)) else v

    cand['__slice__'] = [ _slice_for_sort(dsc, st) for dsc, st in zip(cand['SeriesDescription'], cand.get('SliceThickness', np.nan)) ]
    def _nimg(x):
        try: return int(x)
        except: return -1
    cand['__nimg__']  = cand.get('NumImages', pd.Series([-1]*len(cand))).apply(_nimg)
    def _sno(x):
        try: return int(x)
        except: return 10**9
    cand['__sno__']   = cand.get('SeriesNumber', pd.Series([None]*len(cand))).apply(_sno)

    cand = cand.sort_values(by=['StudyFolder','__score__','__slice__','__nimg__','__sno__'],
                            ascending=[True, False, True, False, True])
    best = cand.groupby('StudyFolder', as_index=False).head(1)

    # Diagnostics
    if verbose:
        n_total = d['StudyFolder'].nunique()
        n_A = candA['StudyFolder'].nunique()
        n_best = best['StudyFolder'].nunique()
        print(f"[Selector] Total studies: {n_total} | Selected in Stage A: {n_A} | Final covered: {n_best}")
        # Show top reasons for initial exclusion
        dropped = d[~A_mask]
        dropped_reasons = {
            'hard_exclude': dropped['SeriesDescription'].apply(is_hard_excluded).sum(),
            'not_likely_coronary': (~dropped['SeriesDescription'].apply(is_likely_coronary)).sum()
        }
        print(f"[Selector] Stage A dropped counts: {dropped_reasons}")
        # Examples for auditing (first few per bucket)
        ex_not_cor = dropped[~dropped['SeriesDescription'].apply(is_likely_coronary)].head(10)['SeriesDescription'].tolist()
        ex_hard = dropped[dropped['SeriesDescription'].apply(is_hard_excluded)].head(10)['SeriesDescription'].tolist()
        print("[Selector] Examples not_likely_coronary:", ex_not_cor)
        print("[Selector] Examples hard_exclude:", ex_hard)

    # Clean temp cols for return
    best_clean = best.drop(columns=[c for c in best.columns if c.startswith('__')], errors='ignore')
    cand_clean = cand.drop(columns=[c for c in cand.columns if c.startswith('__')], errors='ignore')
    return best_clean, cand_clean

# ---- RUN ----
best_per_study, ranked = select_best_ccta_per_study_gentle(df_filtered, hr_hint=None, verbose=True)

# Quick sanity peek
print(best_per_study[['StudyFolder','SeriesDescription']].head(20))


In [ ]:
best_per_study

In [ ]:
best_per_study["StudyFolder"].nunique()

In [ ]:
df_filtered["SeriesDescription"].unique().tolist()

In [ ]:
# Using SimpleITK, visualize df_filtered["RepresentativeFile"].iloc[0]
import os
import SimpleITK as sitk
import matplotlib.pyplot as plt

# ----------------------------
# INPUT: any one DICOM file (representative slice) from df_filtered
# ----------------------------
example_file = df_filtered["RepresentativeFile"].iloc[4]

# ----------------------------
# Helper: read DICOM header (no pixels) with SimpleITK
# ----------------------------
def read_dicom_header(filepath, load_private=True):
    reader = sitk.ImageFileReader()
    reader.SetFileName(filepath)
    if load_private:
        # enable private tag reading where supported
        try:
            reader.LoadPrivateTagsOn()
        except Exception:
            pass
    reader.ReadImageInformation()
    tags = {}
    for key in reader.GetMetaDataKeys():
        try:
            tags[key] = reader.GetMetaData(key)
        except Exception:
            tags[key] = "<unreadable>"
    return tags

header = read_dicom_header(example_file, load_private=True)

# Print header (key -> value)
for key in sorted(header.keys()):
    print(f"{key}: {header[key]}")

# ----------------------------
# QUICK VIEW: show this single file
# ----------------------------
img_2d = sitk.ReadImage(example_file)      # will decode JPEG Lossless etc.
arr_2d = sitk.GetArrayFromImage(img_2d)    # shape: (1, H, W) or (H, W) depending on reader
arr_show = arr_2d[0] if arr_2d.ndim == 3 else arr_2d

plt.figure(figsize=(6,6))
plt.imshow(arr_show, cmap="gray")
plt.title(os.path.basename(example_file))
plt.axis("off")
plt.show()

In [ ]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
from tqdm import tqdm

# You can limit to first N for sanity check, e.g. df.head(10)
files = df_filtered["RepresentativeFile"].iloc[1]

print(f"Rendering {len(files)} representative DICOMs...")

for path in tqdm(files):
    try:
        img = sitk.ReadImage(path)
        arr = sitk.GetArrayFromImage(img)  # shape: (slices, height, width)
        arr2d = arr[0] if arr.ndim == 3 else arr  # take first slice if 3D
        plt.figure(figsize=(5, 5))
        plt.imshow(arr2d, cmap="gray")
        plt.title(path.split('/')[-3])  # e.g. E100**** folder
        plt.axis("off")
        plt.show()
    except Exception as e:
        print(f"[WARN] Could not read {path}: {e}")

In [ ]:
# sort by StudyFolder and ImageType
df_filtered = df_filtered.sort_values(by=['StudyFolder', 'ImageType'])
df_filtered[['StudyFolder', 'ImageType', 'SeriesDescription']].head(5)

In [ ]:
#ContrastBolusAgent
print(df_filtered['ContrastBolusAgent'].value_counts())

In [ ]:
#XRayTubeCurrent
print(df['XRayTubeCurrent'].value_counts())

In [ ]:
#KVP
print(df['KVP'].value_counts())

In [ ]:
df["RepresentativeFile"].iloc[0]

In [ ]:
df["SeriesInstanceUID"].iloc[0]

In [ ]:
import os
import SimpleITK as sitk
import matplotlib.pyplot as plt

# ----------------------------
# CONFIG: point to any one DICOM file from the series
# ----------------------------
example_file = df_filtered["RepresentativeFile"].iloc[0]

# ----------------------------
# Helper: read DICOM header (no pixels) with SimpleITK
# ----------------------------
def read_dicom_header_only(dcm_path: str, load_private: bool = True) -> dict:
    rdr = sitk.ImageFileReader()
    rdr.SetFileName(dcm_path)
    # ReadImageInformation() parses header only
    rdr.ReadImageInformation()
    # enable reading of private tags if requested (supported by GDCM backend inside ITK)
    if load_private:
        try:
            rdr.LoadPrivateTagsOn()
        except Exception:
            pass  # older ITK/SimpleITK may already expose private tags
    meta = {}
    for k in rdr.GetMetaDataKeys():
        try:
            meta[k] = rdr.GetMetaData(k)
        except Exception:
            meta[k] = "<unreadable>"
    return meta

# ----------------------------
# Helper: get SeriesInstanceUID of a given file (via header-only read)
# ----------------------------
def get_series_uid(dcm_path: str) -> str:
    meta = read_dicom_header_only(dcm_path, load_private=True)
    # DICOM tag (0020,000E) = Series Instance UID
    # SimpleITK exposes as "0020|000e" (case-insensitive)
    for key in ("0020|000e", "0020|000E"):
        if key in meta:
            return meta[key]
    raise RuntimeError(f"SeriesInstanceUID not found in {dcm_path}")

# ----------------------------
# Given example file, resolve its series directory and UID
# ----------------------------
series_dir = os.path.dirname(example_file)
series_uid = get_series_uid(example_file)
print("SeriesInstanceUID:", series_uid)
print("Series directory :", series_dir)

# ----------------------------
# Find all series in directory; if not found, try parent
# ----------------------------
r = sitk.ImageSeriesReader()
uids_here = r.GetGDCMSeriesIDs(series_dir) or []
if series_uid not in uids_here:
    parent = os.path.dirname(series_dir)
    uids_parent = r.GetGDCMSeriesIDs(parent) or []
    if series_uid in uids_parent:
        series_dir = parent
        print("Target series found in parent dir:", series_dir)
    else:
        raise RuntimeError(f"Target UID not found.\n Here: {uids_here}\n Parent: {uids_parent}")

# ----------------------------
# Get the file list (sorted) for that series and load volume
# ----------------------------
file_list = r.GetGDCMSeriesFileNames(series_dir, series_uid)
print(f"Found {len(file_list)} slices in series.")

r.SetFileNames(file_list)
# Optional: these controls help expose per-file metadata into the reader if needed
r.MetaDataDictionaryArrayUpdateOn()
r.LoadPrivateTagsOn()

img = r.Execute()  # SimpleITK Image: 3D (z,y,x) for a classic CT series, or multi-frame handled internally
arr = sitk.GetArrayFromImage(img)  # numpy array: shape (slices, H, W)

print("Volume shape (z, y, x):", arr.shape)
print("Voxel spacing (x, y, z) [mm]:", img.GetSpacing())
print("Origin (x, y, z):", img.GetOrigin())
print("Direction (3x3, flattened):", img.GetDirection())

# ----------------------------
# Print key series headers from the first slice file (no external deps)
# ----------------------------
rep_meta = read_dicom_header_only(file_list[0], load_private=True)
def get(meta, tag_hex, default="NA"):
    # tag_hex like "0008|103E" (SeriesDescription)
    return meta.get(tag_hex, default)

print("\n--- Series header (selected tags from first file) ---")
print("SeriesDescription (0008,103E):", get(rep_meta, "0008|103e"))
print("ProtocolName    (0018,1030):", get(rep_meta, "0018|1030"))
print("ImageType       (0008,0008):", get(rep_meta, "0008|0008"))
print("SliceThickness  (0018,0050):", get(rep_meta, "0018|0050"))
print("PixelSpacing    (0028,0030):", get(rep_meta, "0028|0030"))
print("ConvolutionKernel (0018,1210):", get(rep_meta, "0018|1210"))
print("ContrastBolusAgent (0018,0010):", get(rep_meta, "0018|0010"))
print("KVP            (0018,0060):", get(rep_meta, "0018|0060"))
print("StudyDate      (0008,0020):", get(rep_meta, "0008|0020"))
print("SeriesDate     (0008,0021):", get(rep_meta, "0008|0021"))
print("AcqTime        (0008,0032):", get(rep_meta, "0008|0032"))

# If you want to inspect *all* tags for the representative file:
# for k in sorted(rep_meta.keys()):
#     print(k, ":", rep_meta[k])

# ----------------------------
# Visualize a few slices
# ----------------------------
import math
n = arr.shape[0]
for idx in [0, n//3, n//2, (2*n)//3, n-1]:
    plt.figure(figsize=(6,6))
    plt.imshow(arr[idx], cmap="gray")
    plt.title(f"Slice {idx+1}/{n}")
    plt.axis("off")
    plt.show()


In [ ]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
from tqdm import tqdm

# You can limit to first N for sanity check, e.g. df.head(10)
files = df_filtered["RepresentativeFile"].iloc[0]

print(f"Rendering {len(files)} representative DICOMs...")

for path in tqdm(files):
    try:
        img = sitk.ReadImage(path)
        arr = sitk.GetArrayFromImage(img)  # shape: (slices, height, width)
        arr2d = arr[0] if arr.ndim == 3 else arr  # take first slice if 3D
        plt.figure(figsize=(5, 5))
        plt.imshow(arr2d, cmap="gray")
        plt.title(path.split('/')[-3])  # e.g. E100**** folder
        plt.axis("off")
        plt.show()
    except Exception as e:
        print(f"[WARN] Could not read {path}: {e}")
